<a href="https://colab.research.google.com/github/jaimebaldeon/FNS-2020/blob/master/financialStocks_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# prueba investpy
!pip install git+https://github.com/alvarobartt/investpy.git@developer
import investpy


In [1]:

df = investpy.get_stock_historical_data(stock='BABA',
                                        country='United States',
                                        from_date='01/01/2010',
                                        to_date='01/01/2020')
print(df)

NameError: ignored

In [11]:
data = investpy.get_crypto_historical_data(crypto='bitcoin', from_date='01/01/2014', to_date='01/01/2019')
print(data)

              Open    High     Low   Close  Volume Currency
Date                                                       
2014-01-01   805.9   829.9   771.0   815.9   10757      USD
2014-01-02   815.9   886.2   810.5   856.9   12812      USD
2014-01-03   856.9   888.2   839.4   884.3    9709      USD
2014-01-04   884.3   932.2   848.3   924.7   14239      USD
2014-01-05   924.7  1029.9   911.4  1014.7   21374      USD
...            ...     ...     ...     ...     ...      ...
2018-12-28  3587.1  3900.3  3565.5  3861.6  565237      USD
2018-12-29  3861.6  3899.6  3696.0  3706.8  505408      USD
2018-12-30  3708.2  3837.7  3682.5  3815.0  519173      USD
2018-12-31  3815.1  3819.6  3658.8  3709.4  545825      USD
2019-01-01  3709.5  3814.3  3664.4  3809.4  469107      USD

[1827 rows x 6 columns]


In [2]:
import pandas as pd
from bs4 import BeautifulSoup
import urllib.request as ur

In [5]:
def convert_to_int(df): 
  col = df.columns
  for col in df: 
    temp = df[col].to_string().replace(',','').split('\n') #     Replace comma with space and clean out extra spaces
    temp.pop(0) #Remove first erronous element
    df.index = pd.to_datetime(df.index) # Convert index to datetime 
    df[col]= [i[10:].strip() for i in temp] # stripping off nth characters
    df[col] = pd.to_numeric(df[col], errors='coerce').fillna(0).astype(int) # Transform columns from   object to numeric
  return df
  
def clean_data(is_data):
  Fin_st = pd.DataFrame(is_data[0:])
  Fin_st.columns = Fin_st.iloc[0] # Name columns to first row of dataframe
  Fin_st = Fin_st.iloc[1:,] # start to read 1st row
  Fin_st = Fin_st.T # transpose dataframe
  Fin_st.columns = Fin_st.iloc[0] #Name columns to first row of dataframe
  Fin_st.drop(Fin_st.index[0],inplace=True) #Drop first index row
  Fin_st.index.name = '' # Remove the index name
  Fin_st.rename(index={'ttm': '12/31/2019'},inplace=True) #Rename ttm in index columns to end of the year
  #Fin_st = Fin_st[Fin_st.columns[:-5]] # remove last 5 irrelevant columns
  return Fin_st

def extract_data(url, fin_st):
  read_data = ur.urlopen(url).read()
  soup_is = BeautifulSoup(read_data, 'lxml')
  div = []
  for l in soup_is.find_all('div'):
    # Find all divs in site
    #print(l.string)
    if l.string is not None:
      div.append(str(l.string))
  div = list(filter(None, div))
  #print(div[15:])
  div = div[15:]
  # Add missing values in div
  div[0] = 'Annual'
  if fin_st is 'is':
    div = div[:6] + ['Total Revenue'] + div[6:]
    div = div[:24] + ['Operating Expense'] + div[24:]
    div = div[:36] + ['Net Non Operating Interest Income Expense'] + div[36:]
    div = div[:42] + ['Other Income Expense'] + div[42:]
    div = div[:60] + ['Net Income Common Stockholders'] + div[60:]
  elif fin_st is 'bs':
    div = div[:5] + ['Total Assets'] + div[5:]
    div = div[:10] + ['Total Liabilities Net Minority Interest'] + div[10:]
    div = div[:15] + ['Total Equity Gross Minority Interest'] + div[15:]
  elif fin_st is 'cf':
    div = div[:6] + ['Operating Cash Flow'] + div[6:]
    div = div[:12] + ['Investing Cash Flow'] + div[12:]
    div = div[:18] + ['Financing Cash Flow'] + div[18:]
    div = div[:24] + ['End Cash Position'] + div[24:]
  else:
    return False
  #print(new_ls)
  return div

In [3]:
# Stock symbol
index= 'GOLD'

# URL link 
url_is = 'https://finance.yahoo.com/quote/' + index + '/financials?p=' + index
url_bs = 'https://finance.yahoo.com/quote/' + index +'/balance-sheet?p=' + index
url_cf = 'https://finance.yahoo.com/quote/' + index + '/cash-flow?p='+ index
print(url_is)


https://finance.yahoo.com/quote/GOLD/financials?p=GOLD


In [6]:
##  INCOME STATEMENT

new_ls = extract_data(url_is, 'is')
is_data = list(zip(*[iter(new_ls)]*6))
Income_st = clean_data(is_data)
Income_st

Annual,Total Revenue,Cost of Revenue,Gross Profit,Operating Expense,Operating Income,Net Non Operating Interest Income Expense,Other Income Expense,Pretax Income,Tax Provision,Net Income Common Stockholders,Diluted NI Available to Com Stockholders,Basic EPS,Diluted EPS,Basic Average Shares,Diluted Average Shares,Total Expenses,Interest Income,Interest Expense,Net Interest Income,Net Income from Continuing & Discontinued Operation,Normalized Income,EBIT,EBITDA,Reconciled Cost of Revenue,Reconciled Depreciation,Net Income from Continuing Operation Net Minority Interest,Total Unusual Items Excluding Goodwill,Total Unusual Items,Normalized EBITDA,Tax Rate for Calcs,Tax Effect of Unusual Items
,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
12/31/2019,"12,199,000","7,590,000","4,609,000","366,000","4,243,000","-362,000","2,163,000","6,044,000","1,712,000","3,026,000","3,026,000",-,-,-,-,"7,956,000","55,000","351,000","-362,000","3,026,000","2,165,907","6,395,000","8,631,000","7,590,000","2,236,000","3,026,000","1,200,000","1,200,000","7,431,000",0,"339,907"
12/31/2019,"9,717,000","6,911,000","2,806,000","325,000","2,481,000","-468,000","4,344,000","6,357,000","1,783,000","3,969,000","3,969,000",0.0023,0.0023,"1,758,000","1,758,000","7,236,000","51,000","426,000","-468,000","3,969,000","2,855,580","6,783,000",-,"6,911,000","2,032,000","3,969,000","1,687,000","1,687,000","7,128,000",0,"573,580"
12/31/2018,"7,243,000","5,220,000","2,023,000","648,000","1,375,000","-519,000","-1,093,000","-237,000","1,198,000","-1,545,000","-1,545,000",-0.0013,-0.0013,"1,167,000","1,167,000","5,868,000","37,000","442,000","-519,000","-1,545,000","-723,270","205,000",-,"5,220,000","1,457,000","-1,545,000","-1,118,000","-1,118,000","2,780,000",0,"-296,270"
12/31/2017,"8,374,000","5,300,000","3,074,000","657,000","2,417,000","-593,000","923,000","2,747,000","1,231,000","1,438,000","1,438,000",0.0012,0.0012,"1,166,000","1,166,000","5,957,000","14,000","512,000","-593,000","1,438,000","814,720","3,259,000",-,"5,300,000","1,647,000","1,438,000","848,000","848,000","4,058,000",0,"224,720"
12/31/2016,"8,558,000","5,405,000","3,153,000","635,000","2,518,000","-667,000","-73,000","1,778,000","917,000","655,000","655,000",0,0,"1,165,000","1,165,000","6,040,000","13,000","593,000","-667,000","655,000","733,645","2,371,000",-,"5,405,000","1,574,000","655,000","-107,000","-107,000","4,052,000",0,"-28,355"


In [7]:
##  BALANCE SHEET

new_ls = extract_data(url_bs, 'bs')
is_data = list(zip(*[iter(new_ls)]*5))
Balance_sh = clean_data(is_data)
Balance_sh

Annual,Total Assets,Total Liabilities Net Minority Interest,Total Equity Gross Minority Interest,Total Capitalization,Common Stock Equity,Capital Lease Obligations,Net Tangible Assets,Working Capital,Invested Capital,Tangible Book Value,Total Debt,Net Debt,Share Issued,Ordinary Shares Number,Learn more
,,,,,,,,,,,,,,,
12/31/2019,"44,392,000","14,565,000","29,827,000","26,522,000","21,432,000","96,000","16,437,000","4,511,000","26,872,000","16,437,000","5,536,000","2,126,000","1,777,927","1,777,927",Advertise With Us
12/31/2018,"22,631,000","13,246,000","9,385,000","13,280,000","7,593,000","19,000","6,190,000","2,310,000","13,312,000","6,190,000","5,738,000","4,148,000","1,751,982","1,751,982",Advertise With Us
12/31/2017,"25,308,000","14,241,000","11,067,000","15,631,000","9,286,000","46,000","7,701,000","2,937,000","15,663,000","7,701,000","6,423,000","4,143,000","1,166,577","1,166,577",© 2020 Verizon Media. All rights reserved.
12/31/2016,"25,264,000","14,951,000","10,313,000","15,723,000","7,935,000",-,"6,292,000","3,055,000","15,866,000","6,292,000","7,931,000","5,542,000","1,165,574","1,165,574",Discover new investment ideas by accessing unb...


In [8]:
##  CASH FLOW

new_ls = extract_data(url_cf, 'cf')
is_data = list(zip(*[iter(new_ls)]*6))
CashFlow_st = clean_data(is_data)
CashFlow_st

Annual,Operating Cash Flow,Investing Cash Flow,Financing Cash Flow,End Cash Position,Capital Expenditure,Issuance of Debt,Repayment of Debt,Free Cash Flow
,,,,,,,,
12/31/2019,"4,654,000","-519,000","-1,793,000","4,747,000","-1,954,000",-,"-377,000","2,700,000"
12/31/2019,"2,833,000","50,000","-1,139,000","3,314,000","-1,701,000",-,"-309,000","1,132,000"
12/31/2018,"1,765,000","-1,494,000","-925,000","1,571,000","-1,400,000",-,"-687,000","365,000"
12/31/2017,"2,065,000","-337,000","-1,886,000","2,234,000","-1,396,000",0,"-1,533,000","669,000"
12/31/2016,"2,640,000","-412,000","-2,297,000","2,389,000","-1,126,000","5,000","-2,062,000","1,514,000"


In [9]:
convert_to_int(Income_st)

Annual,Total Revenue,Cost of Revenue,Gross Profit,Operating Expense,Operating Income,Net Non Operating Interest Income Expense,Other Income Expense,Pretax Income,Tax Provision,Net Income Common Stockholders,Diluted NI Available to Com Stockholders,Basic EPS,Diluted EPS,Basic Average Shares,Diluted Average Shares,Total Expenses,Interest Income,Interest Expense,Net Interest Income,Net Income from Continuing & Discontinued Operation,Normalized Income,EBIT,EBITDA,Reconciled Cost of Revenue,Reconciled Depreciation,Net Income from Continuing Operation Net Minority Interest,Total Unusual Items Excluding Goodwill,Total Unusual Items,Normalized EBITDA,Tax Rate for Calcs,Tax Effect of Unusual Items
,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2019-12-31,12199000,7590000,4609000,366000,4243000,-362000,2163000,6044000,1712000,3026000,3026000,0,0,0,0,7956000,55000,351000,-362000,3026000,2165907,6395000,8631000,7590000,2236000,3026000,1200000,1200000,7431000,0,339907
2019-12-31,9717000,6911000,2806000,325000,2481000,-468000,4344000,6357000,1783000,3969000,3969000,0,0,1758000,1758000,7236000,51000,426000,-468000,3969000,2855580,6783000,0,6911000,2032000,3969000,1687000,1687000,7128000,0,573580
2018-12-31,7243000,5220000,2023000,648000,1375000,-519000,-1093000,-237000,1198000,-1545000,-1545000,0,0,1167000,1167000,5868000,37000,442000,-519000,-1545000,-723270,205000,0,5220000,1457000,-1545000,-1118000,-1118000,2780000,0,-296270
2017-12-31,8374000,5300000,3074000,657000,2417000,-593000,923000,2747000,1231000,1438000,1438000,0,0,1166000,1166000,5957000,14000,512000,-593000,1438000,814720,3259000,0,5300000,1647000,1438000,848000,848000,4058000,0,224720
2016-12-31,8558000,5405000,3153000,635000,2518000,-667000,-73000,1778000,917000,655000,655000,0,0,1165000,1165000,6040000,13000,593000,-667000,655000,733645,2371000,0,5405000,1574000,655000,-107000,-107000,4052000,0,-28355


In [ ]:
#ratio['effective tax rate']= Income_st['Income Tax Expense']/Income_st['Income Before Tax']